In [1]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

2.5.0
2.5.0


In [2]:
## Load Dataset
import csv
import dataset

# Load class map - these tables map the original TACO classes to your desired class system
# and allow you to discard classes that you don't want to include.
class_map = {}
with open("taco_config/map_10.csv") as csvfile:
    reader = csv.reader(csvfile)
    class_map = {row[0]:row[1] for row in reader}

# Load full dataset or a subset
TACO_DIR = "../data"
round = None # Split number: If None, loads full dataset else if int > 0 selects split no 
subset = "train" # Used only when round !=None, Options: ('train','val','test') to select respective subset
dataset = dataset.Taco()
taco = dataset.load_taco(TACO_DIR, round, subset, class_map=class_map, return_taco=True)

# Must call before using the dataset
dataset.prepare()

print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

ModuleNotFoundError: No module named 'skimage'

In [4]:
from config import Config
class TacoTestConfig(Config):
    NAME = "taco"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.3
    NUM_CLASSES = dataset.num_classes
#     IMAGE_MAX_DIM = 1024
#     IMAGE_MIN_DIM = 1024
#     IMAGE_RESIZE_MODE = "square"
config = TacoTestConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_CLASSLESS_NMS_THRESHOLD 0.9
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.3
DETECTION_NMS_THRESHOLD        0.3
DETECTION_SCORE_RATIO          True
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                23
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MASK_SHARE 

In [6]:
import model as modellib
# Create model objects in inference mode.
# inference mode means we are taking live data points to calculate an output #
# model dir is where the trained model is saved #
# config: exposes a config class, using those settings #
model = modellib.MaskRCNN(mode="inference", model_dir='mask_rcnn_coco.hy', config=config)

# Load weights trained on MS-COCO
# The trained objects that are used for accuracy #
# by_name: weights are loaded into layers only if the share the same name #, exclude=["mrcnn_bbox_fc","mrcnn_class_logits","mrcnn_mask"]
model.load_weights('../mask_rcnn_taco_0100.h5', by_name=True, weights_out_path=None)

In [7]:
print(type(modellib))

<class 'module'>


In [ ]:
%matplotlib inline
import skimage.io
import matplotlib.pyplot as plt
# load an image #
#skimage helps with image processing on a computer #
image = skimage.io.imread('../1.jpg')

# original image
# width and height in inches #
plt.figure(figsize=(12,10))

# show image #
skimage.io.imshow(image)

In [ ]:
class_names = ["BG","Bottle","Bottle cap","Can","Cigarette","Cup","Lid","Other","Plastic bag + wrapper","Pop tab","Straw"]
print(class_names)

In [ ]:
r = model.detect([image], verbose=0)[0]
print(r['class_ids'])

In [ ]:
import visualize
import utils
if r['class_ids'].shape[0]>0:
    r_fused = utils.fuse_instances(r)
else:
    r_fused = r
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
visualize.display_instances(image, r_fused['rois'], r_fused['masks'], r_fused['class_ids'], class_names, r_fused['scores'])
#image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
#visualize.display_instances(image, gt_bbox, gt_mask, gt_class_id, dataset.class_names, title="GT", ax=ax3)